<center>
<img src="https://play-lh.googleusercontent.com/mKhI8xVa-ukFCTPrep10Q7hZ-S6fQFARVA-7jjI-9XhEsQ_WKOnW6ETq7VVNHd_4hj0" width = '200'>

**Data Culture & Governance Challenge | Ignacio Titimoli - may/24**


</center>

Para este ejercicio de análisis de datos, trabajaremos con una base reducida proporcionada por Mercado Libre que contendrá información sobre los productos alocados dentro de la categoría "Joyas y Relojes".
A su vez, circunscribiremos nuestro análisis a la región de Argentina y trabajaremos en el rango de fechas comprendido dentro de los últimos 90 días.

## Generación de scripts de python y normalización de la información obtenida de la API

Comenzaremos por importar las librerías con las que trabajaremos a lo largo de la notebook.

In [1]:
import requests
import pandas as pd
import os
import json
from datetime import datetime
import numpy as np
import sqlite3

A continuación, detallaremos las credenciales necesarias para poder conectarnos a la API de Mercado Libre. Si bien las mismas son de usabilidad temporaria, las dejaremos descriptas debajo porque serán de utilidad a lo largo del código ya sea como variables en sí mismas o como referencias inmediatas.

**Por razones de seguridad, las mismas han sido removidas.**

In [2]:
clientId = ''
secretKey = ''
redirectUrl = 'https://www.movado.com/en/home'
code = ''
access_token = ''
refresh_token = ''

Para la autenticación dentro de Mercado Libre con la API como intermediaria, utilizaremos la URL correspondiente a Movado, una tienda internacional de relojes.

In [ ]:
# Autenticación y autorización

El refresh token permitirá generar las credenciales necesarias para poder establecer la conexión con la API de Mercado Libre. Debajo se muestra cuál es el código que ejecuta el endpoint o punto de enlace para acceder a dicha información.

Consultaremos los códigos de país o `sites` en los cuales Mercado Libre está presente para poder circunscribir nuestro análisis únicamente a la región argentina.

In [ ]:
# Consulta de códigos de país

url = "https://api.mercadolibre.com/sites"

payload = {}
headers = {
  'Authorization': 'Bearer $ACCESS_TOKEN'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

[{"default_currency_id":"DOP","id":"MRD","name":"Dominicana"},{"default_currency_id":"ARS","id":"MLA","name":"Argentina"},{"default_currency_id":"CRC","id":"MCR","name":"Costa Rica"},{"default_currency_id":"BRL","id":"MLB","name":"Brasil"},{"default_currency_id":"CLP","id":"MLC","name":"Chile"},{"default_currency_id":"USD","id":"MSV","name":"El Salvador"},{"default_currency_id":"MXN","id":"MLM","name":"Mexico"},{"default_currency_id":"COP","id":"MCO","name":"Colombia"},{"default_currency_id":"BOB","id":"MBO","name":"Bolivia"},{"default_currency_id":"USD","id":"MEC","name":"Ecuador"},{"default_currency_id":"PAB","id":"MPA","name":"Panamá"},{"default_currency_id":"PEN","id":"MPE","name":"Perú"},{"default_currency_id":"HNL","id":"MHN","name":"Honduras"},{"default_currency_id":"GTQ","id":"MGT","name":"Guatemala"},{"default_currency_id":"UYU","id":"MLU","name":"Uruguay"},{"default_currency_id":"NIO","id":"MNI","name":"Nicaragua"},{"default_currency_id":"VES","id":"MLV","name":"Venezuela"},{

Crearemos un json con la respuesta y la almacenaremos en un dataframe de Python.

In [ ]:
json_data = '[{"default_currency_id":"HNL","id":"MHN","name":"Honduras"},{"default_currency_id":"BOB","id":"MBO","name":"Bolivia"},\
{"default_currency_id":"DOP","id":"MRD","name":"Dominicana"},{"default_currency_id":"MXN","id":"MLM","name":"Mexico"},\
{"default_currency_id":"CRC","id":"MCR","name":"Costa Rica"},{"default_currency_id":"UYU","id":"MLU","name":"Uruguay"},\
{"default_currency_id":"VES","id":"MLV","name":"Venezuela"},{"default_currency_id":"COP","id":"MCO","name":"Colombia"},\
{"default_currency_id":"CLP","id":"MLC","name":"Chile"},{"default_currency_id":"USD","id":"MEC","name":"Ecuador"},\
{"default_currency_id":"USD","id":"MSV","name":"El Salvador"},{"default_currency_id":"ARS","id":"MLA","name":"Argentina"},\
{"default_currency_id":"PYG","id":"MPY","name":"Paraguay"},{"default_currency_id":"PEN","id":"MPE","name":"Perú"},\
{"default_currency_id":"NIO","id":"MNI","name":"Nicaragua"},{"default_currency_id":"PAB","id":"MPA","name":"Panamá"},\
{"default_currency_id":"CUP","id":"MCU","name":"Cuba"},{"default_currency_id":"BRL","id":"MLB","name":"Brasil"},\
{"default_currency_id":"GTQ","id":"MGT","name":"Guatemala"}]'

response_df = pd.read_json(json_data)

response_df.sort_values(by='name').head()

,default_currency_id,id,name
11,ARS,MLA,Argentina
1,BOB,MBO,Bolivia
17,BRL,MLB,Brasil
8,CLP,MLC,Chile
7,COP,MCO,Colombia


Como podemos ver, el ID por medio del cual se referencia a Argentina es `MLA`.

A continuación listaremos, entonces, las categorías disponibles dentro de Mercado Libre Argentina (MLA), almacenando también la respuesta en otro dataframe de Python, que llamaremos `category`.

In [ ]:
# Categorías disponibles para Arg

url = "https://api.mercadolibre.com/sites/MLA/categories"

payload = {}
headers = {
  'Authorization': 'Bearer $ACCESS_TOKEN'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


[{"id":"MLA5725","name":"Accesorios para Vehículos"},{"id":"MLA1512","name":"Agro"},{"id":"MLA1403","name":"Alimentos y Bebidas"},{"id":"MLA1071","name":"Animales y Mascotas"},{"id":"MLA1367","name":"Antigüedades y Colecciones"},{"id":"MLA1368","name":"Arte, Librería y Mercería"},{"id":"MLA1743","name":"Autos, Motos y Otros"},{"id":"MLA1384","name":"Bebés"},{"id":"MLA1246","name":"Belleza y Cuidado Personal"},{"id":"MLA1039","name":"Cámaras y Accesorios"},{"id":"MLA1051","name":"Celulares y Teléfonos"},{"id":"MLA1648","name":"Computación"},{"id":"MLA1144","name":"Consolas y Videojuegos"},{"id":"MLA1500","name":"Construcción"},{"id":"MLA1276","name":"Deportes y Fitness"},{"id":"MLA5726","name":"Electrodomésticos y Aires Ac."},{"id":"MLA1000","name":"Electrónica, Audio y Video"},{"id":"MLA2547","name":"Entradas para Eventos"},{"id":"MLA407134","name":"Herramientas"},{"id":"MLA1574","name":"Hogar, Muebles y Jardín"},{"id":"MLA1499","name":"Industrias y Oficinas"},{"id":"MLA1459","name":"I

In [3]:
json_category = '[{"id":"MLA5725","name":"Accesorios para Vehículos"},{"id":"MLA1512","name":"Agro"},{"id":"MLA1403","name":"Alimentos y Bebidas"},{"id":"MLA1071","name":"Animales y Mascotas"},{"id":"MLA1367","name":"Antigüedades y Colecciones"},{"id":"MLA1368","name":"Arte, Librería y Mercería"},{"id":"MLA1743","name":"Autos, Motos y Otros"},{"id":"MLA1384","name":"Bebés"},{"id":"MLA1246","name":"Belleza y Cuidado Personal"},{"id":"MLA1039","name":"Cámaras y Accesorios"},{"id":"MLA1051","name":"Celulares y Teléfonos"},{"id":"MLA1648","name":"Computación"},{"id":"MLA1144","name":"Consolas y Videojuegos"},{"id":"MLA1500","name":"Construcción"},{"id":"MLA1276","name":"Deportes y Fitness"},{"id":"MLA5726","name":"Electrodomésticos y Aires Ac."},{"id":"MLA1000","name":"Electrónica, Audio y Video"},{"id":"MLA2547","name":"Entradas para Eventos"},{"id":"MLA407134","name":"Herramientas"},{"id":"MLA1574","name":"Hogar, Muebles y Jardín"},{"id":"MLA1499","name":"Industrias y Oficinas"},{"id":"MLA1459","name":"Inmuebles"},{"id":"MLA1182","name":"Instrumentos Musicales"},{"id":"MLA3937","name":"Joyas y Relojes"},{"id":"MLA1132","name":"Juegos y Juguetes"},{"id":"MLA3025","name":"Libros, Revistas y Comics"},{"id":"MLA1168","name":"Música, Películas y Series"},{"id":"MLA1430","name":"Ropa y Accesorios"},{"id":"MLA409431","name":"Salud y Equipamiento Médico"},{"id":"MLA1540","name":"Servicios"},{"id":"MLA9304","name":"Souvenirs, Cotillón y Fiestas"},{"id":"MLA1953","name":"Otras categorías"}]'

category = pd.read_json(json_category)

category.sort_values(by='name').head()

,id,name
0,MLA5725,Accesorios para Vehículos
1,MLA1512,Agro
2,MLA1403,Alimentos y Bebidas
3,MLA1071,Animales y Mascotas
4,MLA1367,Antigüedades y Colecciones


In [ ]:
category.name.tolist()

['Accesorios para Vehículos',
 'Agro',
 'Alimentos y Bebidas',
 'Animales y Mascotas',
 'Antigüedades y Colecciones',
 'Arte, Librería y Mercería',
 'Autos, Motos y Otros',
 'Bebés',
 'Belleza y Cuidado Personal',
 'Cámaras y Accesorios',
 'Celulares y Teléfonos',
 'Computación',
 'Consolas y Videojuegos',
 'Construcción',
 'Deportes y Fitness',
 'Electrodomésticos y Aires Ac.',
 'Electrónica, Audio y Video',
 'Entradas para Eventos',
 'Herramientas',
 'Hogar, Muebles y Jardín',
 'Industrias y Oficinas',
 'Inmuebles',
 'Instrumentos Musicales',
 'Joyas y Relojes',
 'Juegos y Juguetes',
 'Libros, Revistas y Comics',
 'Música, Películas y Series',
 'Ropa y Accesorios',
 'Salud y Equipamiento Médico',
 'Servicios',
 'Souvenirs, Cotillón y Fiestas',
 'Otras categorías']

Dadas todas las categorías, elegiremos trabajar con la ya mencionada "Joyas y Relojes". Buscaremos el ID de la categoría dentro de nuestro dataframe.

In [ ]:
category[category['name'] == 'Joyas y Relojes']

,id,name
23,MLA3937,Joyas y Relojes


Una vez elegida la categoría, nos centraremos en buscar los ítems o productos cargados dentro de ella. Para ello, utilizaremos el recurso "Items by category", con el objetivo de conseguir toda la información vinculada a dichos productos que serán la fuente de nuestro análisis posterior.
Limitaremos nuestra búsqueda a 50 items.

Solicitaremos a través de la API la respuesta. Dada la numerosa cantidad de registros que la misma arroja, generaremos un json externo que importaremos y guardaremos en un nuevo dataframe de Python, llamado `sellers`.

In [4]:
pd.set_option('display.max_columns', None)

json_data = '/content/sellers.json'

with open(json_data, 'r') as f:
    data = json.load(f)

sellers = pd.json_normalize(data['results'] , sep ='_')

sellers.head(5)

,id,title,condition,thumbnail_id,catalog_product_id,listing_type_id,permalink,buying_mode,site_id,category_id,domain_id,thumbnail,currency_id,order_backend,price,original_price,sale_price,available_quantity,official_store_id,official_store_name,use_thumbnail_id,accepts_mercadopago,stop_time,attributes,winner_item_id,catalog_listing,discounts,promotions,inventory_id,shipping_store_pick_up,shipping_free_shipping,shipping_logistic_type,shipping_mode,shipping_tags,shipping_benefits,shipping_promise,seller_id,seller_nickname,installments_quantity,installments_amount,installments_rate,installments_currency_id,installments,variation_filters,variations_data_176310005259_thumbnail,variations_data_176310005259_ratio,variations_data_176310005259_name,variations_data_176310005259_pictures_qty,variations_data_176310005259_price,variations_data_176310005259_inventory_id,variations_data_176310005259_user_product_id,variations_data_176310005259_attributes,variations_data_176310005258_thumbnail,variations_data_176310005258_ratio,variations_data_176310005258_name,variations_data_176310005258_pictures_qty,variations_data_176310005258_price,variations_data_176310005258_inventory_id,variations_data_176310005258_user_product_id,variations_data_176310005258_attributes,variations_data_176556225210_thumbnail,variations_data_176556225210_ratio,variations_data_176556225210_name,variations_data_176556225210_pictures_qty,variations_data_176556225210_price,variations_data_176556225210_inventory_id,variations_data_176556225210_user_product_id,variations_data_176556225210_attributes,differential_pricing_id,variations_data_180400854093_thumbnail,variations_data_180400854093_ratio,variations_data_180400854093_name,variations_data_180400854093_pictures_qty,variations_data_180400854093_price,variations_data_180400854093_inventory_id,variations_data_180400854093_user_product_id,variations_data_180400854093_attributes,variations_data_89101748820_thumbnail,variations_data_89101748820_ratio,variations_data_89101748820_name,variations_data_89101748820_pictures_qty,variations_data_89101748820_price,variations_data_89101748820_user_product_id,variations_data_89101748820_attributes,variations_data_82450487916_thumbnail,variations_data_82450487916_ratio,variations_data_82450487916_name,variations_data_82450487916_pictures_qty,variations_data_82450487916_price,variations_data_82450487916_user_product_id,variations_data_82450487916_attributes,variations_data_82450487920_thumbnail,variations_data_82450487920_ratio,variations_data_82450487920_name,variations_data_82450487920_pictures_qty,variations_data_82450487920_price,variations_data_82450487920_user_product_id,variations_data_82450487920_attributes,variations_data_82450487925_thumbnail,variations_data_82450487925_ratio,variations_data_82450487925_name,variations_data_82450487925_pictures_qty,variations_data_82450487925_price,variations_data_82450487925_user_product_id,variations_data_82450487925_attributes,variations_data_82450487933_thumbnail,variations_data_82450487933_ratio,variations_data_82450487933_name,variations_data_82450487933_pictures_qty,variations_data_82450487933_price,variations_data_82450487933_user_product_id,variations_data_82450487933_attributes,variations_data_82450487945_thumbnail,variations_data_82450487945_ratio,variations_data_82450487945_name,variations_data_82450487945_pictures_qty,variations_data_82450487945_price,variations_data_82450487945_user_product_id,variations_data_82450487945_attributes,variations_data_82450487909_thumbnail,variations_data_82450487909_ratio,variations_data_82450487909_name,variations_data_82450487909_pictures_qty,variations_data_82450487909_price,variations_data_82450487909_user_product_id,variations_data_82450487909_attributes,variations_data_63695796183_thumbnail,variations_data_63695796183_ratio,variations_data_63695796183_name,variations_data_63695796183_pictures_qty,variations_data_63695796183_price,variations_data_63695796183_user_product_id,variations_data_63695

Como puede verse, nuestro dataframe `sellers` contiene 50 filas y 140 columnas. Sin embargo, algunas de las columnas podrían desagregarse aún más (como ser los atributos asociados al ID).

In [ ]:
sellers.shape

(50, 140)

Realizaremos, a su vez, un análisis simple de las primeras cinco empresas agrupadas dentro de estos cincuenta productos, sumarizando la cantidad de productos disponibles y la suma de su precio de venta.

Cabe destacar que los valores asociados a las cantidades, según se expresa en la documentación, son aproximados y que valores como 500 o 600 significan más de 500 o más de 600, respectivamente, y no se trata necesariamente de valores exactos.

Podemos ver que 'Netshop_1000' es la empresa para la que más productos tenemos referenciados en nuestra bajada inicial, aunque ya veremos de qué productos se trata y si los mismos pueden o no ser analizados de forma comparativa con el resto de los que figuran en nuestro dataframe.

In [ ]:
sellers.groupby(['seller_id', 'seller_nickname']).agg({'available_quantity':'sum', 'price': 'sum'}).sort_values(by='available_quantity', ascending=False).head()

,,available_quantity,price
seller_id,seller_nickname,,
139673546,NETSHOP_1000,3251,381440.64
511139715,FORCELATIGROUPSA,600,36348.00
19876329,STIDIGITAL.,500,28549.50
322192254,TIENDAGEKO,500,1579.85
195393276,LULA ACCESORIOS,500,2422.16


Tomaremos de nuestro dataframe `sellers` solo aquellos campos que consideramos más relevantes, desestimando muchos otros que no formarán parte de este análisis.

In [5]:
sellers = sellers[['id', 'title', 'listing_type_id', 'permalink', 'category_id', 'currency_id', 'price', 'original_price', 'sale_price',\
                      'available_quantity', 'official_store_name', 'stop_time', 'attributes', 'seller_id', \
                      'seller_nickname', 'installments_quantity', 'installments_amount', \
                      'installments_currency_id']]

Analizaremos la distribución de nuestro dataframe por medio de la feature `category_id`, que es una subcategoría específica dentro de nuestra categoría principal preseleccionada.

La idea con esto es agrupar dentro de nuestra tabla un análisis sobre un grupo de vendedores que ofrezcan productos de características similares. El objetivo final será realizar la comparación "en igualdad de condiciones".

In [ ]:
sellers.groupby(['category_id', 'seller_id'])[['id']].count().sort_values(by=['category_id', 'id'])

id
category_id seller_id     
MLA1432     322192254    1
            501615772    1
MLA1442     31452355     1
            83989797     1
            99179853     1
            250551537    1
            265893742    1
            269998830    1
            511139715    1
            570612785    1
            1210552456   1
            1619856464   1
            1734129189   1
            31568718     2
            37133205     2
            297697151    2
            39144101     3
            47316577     3
            139673546   12
MLA352679   19876329     1
            257077964    1
            320844861    1
            28785841     2
            37133205     2
MLA4559     75694306     1
            511139715    1
MLA457383   114279911    1
            195393276    1
MLA6312     114279911    1
            243234256    1

La mayoría de nuestros ítems están vinculados al `category_id` = 'MLA1442'. Una de las empresas (`seller_id` = 139673546) ha quedado circunscripta en la barrida inicial con muchos más productos que el resto (12 vs 1, 2 o 3 que ofrece la mayoría).

Tomaremos para continuar con nuestro ejercicio solo aquellos `seller_id` con solo un `item_id` dentro de nuestro dataframe por la misma razón que mencionamos previamente: equiparar lo más posible la aleatoriedad de nuestra muestra.

In [6]:
vend = [31452355, 83989797, 99179853, 250551537, 265893742, 269998830, 511139715, 570612785, 1210552456, \
        1619856464, 1734129189]

sellers = sellers[(sellers['category_id'] == 'MLA1442') & (sellers['seller_id'].isin(vend))]
sellers.groupby(['id', 'seller_id']).agg({'available_quantity':'sum', 'price': 'sum'})

print(sellers.shape)

(11, 18)


Nuestro dataframe ahora solo tiene 11 filas (las de los 11 `seller_id` con los que trabajaremos) y 18 columnas (aquellas que hemos considerado significativas para nuestro análisis).

Buscaremos a continuación el nombre concreto de la categoría con la que estamos trabajando, dado que solo tenemos su ID. Para ello, deberemos conectarnos nuevamente a la API de Mercado Libre.

In [ ]:
url = "https://api.mercadolibre.com/categories/MLA1442"

payload = {}
headers = {
  'Authorization': 'Bearer $ACCESS_TOKEN'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)


{"id":"MLA1442","name":"Relojes Pulsera","picture":"https://http2.mlstatic.com/storage/categories-api/images/786edffb-84e9-4583-9ad5-dd660967b3c7.png","permalink":null,"total_items_in_this_category":79085,"path_from_root":[{"id":"MLA3937","name":"Joyas y Relojes"},{"id":"MLA431677","name":"Relojes"},{"id":"MLA1442","name":"Relojes Pulsera"}],"children_categories":[],"attribute_types":"variations","settings":{"adult_content":false,"buying_allowed":true,"buying_modes":["buy_it_now","auction"],"catalog_domain":"MLA-WRISTWATCHES","coverage_areas":"not_allowed","currencies":["ARS"],"fragile":false,"immediate_payment":"required","item_conditions":["used","not_specified","new"],"items_reviews_allowed":false,"listing_allowed":true,"max_description_length":50000,"max_pictures_per_item":12,"max_pictures_per_item_var":10,"max_sub_title_length":70,"max_title_length":60,"max_variations_allowed":100,"maximum_price":null,"maximum_price_currency":"ARS","minimum_price":1000,"minimum_price_currency":"AR

Almacenaremos nuestros resultados en un nuevo dataframe llamado `subcategory`.
El nombre de nuestra subcategoría es "Relojes Pulsera".

In [7]:
json_subcategory = '[{"id":"MLA1442","name":"Relojes Pulsera","picture":"https://http2.mlstatic.com/storage/categories-api/images/786edffb-84e9-4583-9ad5-dd660967b3c7.png","permalink":null,"total_items_in_this_category":78361,"path_from_root":[{"id":"MLA3937","name":"Joyas y Relojes"},{"id":"MLA431677","name":"Relojes"},{"id":"MLA1442","name":"Relojes Pulsera"}],"children_categories":[],"attribute_types":"variations","settings":{"adult_content":false,"buying_allowed":true,"buying_modes":["buy_it_now","auction"],"catalog_domain":"MLA-WRISTWATCHES","coverage_areas":"not_allowed","currencies":["ARS"],"fragile":false,"immediate_payment":"required","item_conditions":["used","not_specified","new"],"items_reviews_allowed":false,"listing_allowed":true,"max_description_length":50000,"max_pictures_per_item":12,"max_pictures_per_item_var":10,"max_sub_title_length":70,"max_title_length":60,"max_variations_allowed":100,"maximum_price":null,"maximum_price_currency":"ARS","minimum_price":1000,"minimum_price_currency":"ARS","mirror_category":null,"mirror_master_category":null,"mirror_slave_categories":[],"price":"required","reservation_allowed":"not_allowed","restrictions":[],"rounded_address":false,"seller_contact":"not_allowed","shipping_options":["custom","carrier"],"shipping_profile":"optional","show_contact_information":false,"simple_shipping":"optional","stock":"required","sub_vertical":"fashion_accesories","subscribable":false,"tags":[],"vertical":"apparel","vip_subdomain":"articulo","buyer_protection_programs":["delivered","undelivered"],"status":"enabled"},"channels_settings":[{"channel":"mshops","settings":{"minimum_price":0}},{"channel":"proximity","settings":{"status":"disabled"}},{"channel":"mp-merchants","settings":{"buying_modes":["buy_it_now"],"immediate_payment":"required","minimum_price":1,"status":"enabled"}},{"channel":"mp-link","settings":{"buying_modes":["buy_it_now"],"immediate_payment":"required","minimum_price":1,"status":"enabled"}}],"meta_categ_id":null,"attributable":false,"date_created":"2018-04-25T08:12:56.000Z"}]'

subcategory = pd.read_json(json_subcategory)

display(subcategory)

,id,name,picture,permalink,total_items_in_this_category,path_from_root,children_categories,attribute_types,settings,channels_settings,meta_categ_id,attributable,date_created
0,MLA1442,Relojes Pulsera,https://http2.mlstatic.com/storage/categories-...,NaN,78361,"[{'id': 'MLA3937', 'name': 'Joyas y Relojes'},...",[],variations,"{'adult_content': False, 'buying_allowed': Tru...","[{'channel': 'mshops', 'settings': {'minimum_p...",NaN,False,2018-04-25T08:12:56.000Z


Nos quedaremos únicamente con el `id` y el `name` y mergearemos con nuestro dataframe `sellers`, que ahora tendrá 19 columnas en lugar de 18.

In [8]:
subcategory = subcategory[['id', 'name']]
subcategory = subcategory.rename(columns={'id': 'category_id'})
sellers = sellers.merge(subcategory, on='category_id', how='inner')
sellers.shape
#subcategory.head()

(11, 19)

Ahora trabajaremos con la feature `attributes`. Como hemos mencionado, la columna tiene una longitud considerable y no fue desagregada desde un comienzo, como sí ocurrió con casi todo el input del archivo `sellers.json`.

Nos focalizaremos en algunos atributos puntuales, preferentemente en aquellos que estén presentes para todos los productos con los que trabajaremos.

Para ello, comenzaremos por generar un nuevo dataframe, que llamaremos `attributes`. El mismo solo contendrá información del `item_id` y de los `attributes` propiamente dichos.

In [9]:
attributes = sellers[['id', 'attributes']]
attributes.shape

(11, 2)

Llevaremos primero el archivo a un .csv para poder armar por fuera de esta notebook el json file.

In [ ]:
attributes.to_csv('/content/attributes.csv')

Importaremos el archivo de atributos para construir nuestro dataframe de Pandas.

El mismo es de 180x13.

In [10]:
json_data = '/content/attrib.json'

with open(json_data, 'r') as f:
    data = json.load(f)

attributes = pd.json_normalize(data['results'] , sep ='_')

attributes.head(2)

,item_id,id,name,value_id,value_name,attribute_group_id,attribute_group_name,value_struct,values,source,value_type,value_struct_number,value_struct_unit
0,MLA1240020760,AGE_GROUP,Edad,6725189,Adultos,OTHERS,Otros,NaN,"[{'id': '6725189', 'name': 'Adultos', 'struct'...",2579503448603610,list,NaN,NaN
1,NaN,BACKGROUND_COLOR,Color del fondo,52049,Negro,OTHERS,Otros,NaN,"[{'id': '52049', 'name': 'Negro', 'struct': No...",1,string,NaN,NaN


In [11]:
attributes.shape

(180, 13)

Completaremos el dataframe con el valor del `item_id` en aquellas celdas que figuran vacías. Tomaremos los primeros 30 registros del dataframe para confirmar la modificación.

In [12]:
attributes = attributes.fillna(method='ffill')
attributes.head(25)

,item_id,id,name,value_id,value_name,attribute_group_id,attribute_group_name,value_struct,values,source,value_type,value_struct_number,value_struct_unit
0,MLA1240020760,AGE_GROUP,Edad,6725189,Adultos,OTHERS,Otros,NaN,"[{'id': '6725189', 'name': 'Adultos', 'struct'...",2579503448603610,list,NaN,NaN
1,MLA1240020760,BACKGROUND_COLOR,Color del fondo,52049,Negro,OTHERS,Otros,NaN,"[{'id': '52049', 'name': 'Negro', 'struct': No...",1,string,NaN,NaN
2,MLA1240020760,BEZEL_COLOR,Color del bisel,52049,Negro,OTHERS,Otros,NaN,"[{'id': '52049', 'name': 'Negro', 'struct': No...",1,string,NaN,NaN
3,MLA1240020760,BRAND,Marca,120788,Skmei,OTHERS,Otros,NaN,"[{'id': '120788', 'name': 'Skmei', 'struct': N...",1,string,NaN,NaN
4,MLA1240020760,CASE_COLOR,Color de la caja,52049,Negro,OTHERS,Otros,NaN,"[{'id': '52049', 'name': 'Negro', 'struct': No...",1,string,NaN,NaN
5,MLA1240020760,DETAILED_MODEL,Modelo detallado,11933032,SKMEI-9185-WHITE,OTHERS,Otros,NaN,"[{'id': '11933032', 'name': 'SKMEI-9185-WHITE'...",1,string,NaN,NaN
6,MLA1240020760,DIAL_HOURS_COLOR,Color del dial horario,52049,Negro,OTHERS,Otros,NaN,"[{'id': '52049', 'name': 'Negro', 'struct': No...",1,string,NaN,NaN
7,MLA1240020760,DIAL_MINUTES_SECONDS_COLOR,Color del dial del minutero/segundero,59784,Negro/Blanco,OTHERS,Otros,NaN,"[{'id': '59784', 'name': 'Negro/Blanco', 'stru...",1,string,NaN,NaN
8,MLA1240020760,FILTRABLE_GENDER,Género filtrables,18549360,Hombre,OTHERS,Otros,NaN,"[{'id': '18549360', 'name': 'Hombre', 'struct'...",2579503448603610,list,NaN,NaN
9,MLA1240020760,GENDER,Género,339666,Hombre,OTHERS,Otros,NaN,"[{'id': '339666', 'name': 'Hombre', 'struct': ...",3045741222775799,list,NaN,NaN


Finalmente, agruparemos nuestros resultados para ver qué atributos están presentes para todos nuestros `item_id`.

Puede verse que `weight`, `gender`, `brand`, `model` e `item_condition` son los que están presentes para todos los productos.

En principio, nos quedaremos únicamente con estos valores en nuestro dataframe `attributes`.

In [13]:
atr = attributes[['item_id', 'id']]
atr.groupby('id')[['item_id']].count().sort_values(by='item_id', ascending=False)

,item_id
id,
WEIGHT,11
GENDER,11
BRAND,11
MODEL,11
ITEM_CONDITION,11
STRAP_COLOR,10
DETAILED_MODEL,10
FILTRABLE_GENDER,10
BACKGROUND_COLOR,9


In [14]:
atributos = ['WEIGHT', 'GENDER', 'BRAND', 'MODEL', 'ITEM_CONDITION']
attributes = attributes[attributes['id'].isin(atributos)]
attributes.shape

(55, 13)

Ya tenemos nuestro dataframe semi-normalizado. Lo único que resta es:

- Eliminar las columnas que no son de nuestro interés.
- Reacomodar los valores de modo de tener solo 11 filas en nuestro dataframe (correspondientes a los 11 productos) y los atributos de nuestros productos dispuestos en columnas.

In [15]:
attributes = attributes[['item_id', 'name', 'value_name']]

Generaremos una tabla pivot y resetearemos el índice, de modo de poder reacomodar las columnas.
Luego renombraremos las columnas, de modo tal que los valores de la columna `name` figuren asociados a los valores de la columna `value_name`.

In [16]:
attributes = attributes.pivot(index='item_id', columns='name', values='value_name').reset_index()
attributes.columns.name = None
attributes.columns = ['id', 'Condicion del item', 'Genero', 'Marca', 'Modelo', 'Peso']

display(attributes.head())
print(attributes.shape)

,id,Condicion del item,Genero,Marca,Modelo,Peso
0,MLA1240020760,Nuevo,Hombre,Skmei,9185,66 g
1,MLA1361794043,Nuevo,Hombre,Skmei,1068,65 g
2,MLA1367061911,Nuevo,Mujer,WickPro.,Nube,41.5 g
3,MLA1395639417,Nuevo,Hombre,Casio,EFR-552D-1A3,157 g
4,MLA1396892459,Nuevo,Sin género,Casio,A159,44 g


(11, 6)


Podemos ver que nuestro dataframe tiene, como esperábamos, 11 filas y 6 columnas.

Procederemos a mergear este dataframe con nuestra tabla principal: `sellers`.

Nuestro dataframe principal ahora debería tener 11 filas y 24 columnas.

In [17]:
sellers = sellers.merge(attributes, on='id', how='inner')
sellers.shape

(11, 24)

Efectivamente esto es así: la tabla ahora tiene 11 filas y 24 columnas con el agregado de los cinco campos correspondientes a los atributos de nuestros productos.

Ya tenemos consolidados todos nuestros datos en un único dataframe.

Nos resta calcular las ventas que tuvieron cada uno de nuestros productos.
Dado que no existe un dato de ventas, el mismo tendrá que ser inferido.
Para comenzar con este proceso de inferencia de ventas, solicitaremos a la API de Mercado Libre la cantidad de visitas que tuvo cada uno de nuestros productos en los últimos 90 días. Con esa información, construiremos un valor que surgirá por multiplicar la cantidad de visitas por un **valor arbitrario** que setearemos en el 7% para todos los `listing_types` reconocidos como "gold_pro" y del 5% para todos aquellos identificados como "gold_special". De acuerdo a la documentación, las publicaciones de vendedores "gold_pro" ofrecen "cuotas más convenientes a los compradores", por lo que la propensión a la venta debería ser mayor que la de aquellos productos categorizados como "gold_special".

Hay algunos puntos que convendría destacar antes de avanzar:

1 - De haber algún producto con un precio diferencial producto de alguna bonificación ofrecida por el vendedor (diferencia entre `price` y `original_price`), siempre consideraremos la feature `price`, es decir, la que podría contener alguna bonificación, aún cuando ese producto haya tenido la bonificación solo durante un período acotado de tiempo dentro de los 90 días que analizaremos. Esto ocurre porque no contamos con un detalle de la manera en que el vendedor manejó las bonificaciones y creemos no tener manera de poder inferirlo.

2 - Calcularemos un ratio de ventas mensual. De este modo, en 90 días calcularemos 3 ratios: uno para el primer mes, otro para el segundo y otro para el tercero. En este sentido, consideraremos tanto los precios como la cantidad de productos disponibles como flat, esto es: sin modificaciones. La razón es la misma que en el punto anterior: no contamos con información pormenorizada al respecto y creemos, una vez más, no tener manera de poder inferirla.

3 - Ni los costos por publicar ni cualquier otro costo asociado a la venta a través del sitio será considerado a los efectos de este análisis.

4 - En caso que el valor del ratio sea más alto que la cantidad de productos disponibles para cada ítem, se considerará la cantidad de productos disponibles (`available_quantity`).

Con todo esto, armaremos un nuevo dataframe que llamaremos `sales`, y que luego será cruzado con nuestra tabla principal `sellers`.

Detallaremos los id que deberemos consultar a través de la API.

In [ ]:
sellers.id.tolist()

['MLA1240020760',
 'MLA1660923058',
 'MLA1618134820',
 'MLA1702569300',
 'MLA1361794043',
 'MLA1395639417',
 'MLA1396892459',
 'MLA1731413510',
 'MLA1367061911',
 'MLA1744842938',
 'MLA1622811266']

Generaremos una corrida individual para cada `item_id` y guardaremos el resultado en un json file. A efectos de mostrar el procedimiento, dejaremos la instancia de corrida para el primer `item_id`.

[{"item_id":"MLA1622811266","date_from":"2024-02-01T00:00:00Z","date_to":"2024-02-29T00:00:00Z","total_visits":0,"visits_detail":[]}]


[{"item_id":"MLA1622811266","date_from":"2024-03-01T00:00:00Z","date_to":"2024-03-31T00:00:00Z","total_visits":74,"visits_detail":[{"company":"mercadolibre","quantity":74}]}]


[{"item_id":"MLA1622811266","date_from":"2024-04-01T00:00:00Z","date_to":"2024-04-30T00:00:00Z","total_visits":1340,"visits_detail":[{"company":"mercadolibre","quantity":1340}]}]


Importaremos nuestro json file. Nuestra tabla de ventas tiene 33 filas (una por cada uno de nuestros 11 `item_id` y por cada mes de análisis). Las visitas totales (al cabo de 3 meses) fueron 48.490.

In [18]:
json_data = '/content/sales.json'

with open(json_data, 'r') as f:
    data = json.load(f)

sales = pd.json_normalize(data['results'] , sep ='_')

display(sales.head(2))
print(f"La cantidad de visitas al cabo de 3 meses fue de:", sales.total_visits.sum())
print(sales.shape)

,item_id,date_from,date_to,total_visits,visits_detail
0,MLA1240020760,2024-02-01T00:00:00Z,2024-02-29T00:00:00Z,4402,"[{'company': 'mercadolibre', 'quantity': 4402}]"
1,MLA1240020760,2024-03-01T00:00:00Z,2024-03-31T00:00:00Z,4805,"[{'company': 'mercadolibre', 'quantity': 4805}]"


La cantidad de visitas al cabo de 3 meses fue de: 48490
(33, 5)


Modificaremos el formato de la fecha y generaremos una nueva variable con el mes correspondiente.

In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   item_id        33 non-null     object
 1   date_from      33 non-null     object
 2   date_to        33 non-null     object
 3   total_visits   33 non-null     int64 
 4   visits_detail  33 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.4+ KB


In [19]:
sales['date_from'] = pd.to_datetime(sales.date_from)
sales['date_to'] = pd.to_datetime(sales.date_to)
sales['month'] = sales['date_from'].dt.strftime('%B')

sales.head()

,item_id,date_from,date_to,total_visits,visits_detail,month
0,MLA1240020760,2024-02-01 00:00:00+00:00,2024-02-29 00:00:00+00:00,4402,"[{'company': 'mercadolibre', 'quantity': 4402}]",February
1,MLA1240020760,2024-03-01 00:00:00+00:00,2024-03-31 00:00:00+00:00,4805,"[{'company': 'mercadolibre', 'quantity': 4805}]",March
2,MLA1240020760,2024-04-01 00:00:00+00:00,2024-04-30 00:00:00+00:00,789,"[{'company': 'mercadolibre', 'quantity': 789}]",April
3,MLA1660923058,2024-02-01 00:00:00+00:00,2024-02-29 00:00:00+00:00,1949,"[{'company': 'mercadolibre', 'quantity': 1949}]",February
4,MLA1660923058,2024-03-01 00:00:00+00:00,2024-03-31 00:00:00+00:00,2975,"[{'company': 'mercadolibre', 'quantity': 2975}]",March


In [ ]:
sales.groupby(['item_id', 'month'])[['total_visits']].sum().head()

total_visits
item_id       month                 
MLA1240020760 April              789
              February          4402
              March             4805
MLA1361794043 April             2179
              February           109

Podemos ver que el `item_id` "MLA1240020760" fue el que más visitas tuvo en los últimos 90 días, con 9.996.

In [ ]:
sales.groupby('item_id')[['total_visits']].sum().sort_values(by='total_visits', ascending=False).head()

,total_visits
item_id,
MLA1240020760,9996
MLA1367061911,8039
MLA1660923058,7589
MLA1702569300,5741
MLA1395639417,5693


Ahora, calcularemos los ratios, que serán la base para inferir las ventas de cada producto.

Para ello, tomaremos solo aquellas celdas indispensables de nuestro dataframe `sales` para cruzarlas con nuestro dataframe principal, `sellers`.

In [20]:
sales = sales[['item_id', 'total_visits', 'month']]
sales = sales.rename(columns={'item_id': 'id'})
sales = sales.pivot(index='id', columns='month', values='total_visits').reset_index()
sales = sales[['id', 'February', 'March', 'April']]
sales.shape

(11, 4)

Ahora nuestra tabla principal tiene 27 features.

Controlaremos que la moneda, en todos los casos, sea pesos (ARS) y procederemos al cálculo de los ratios de inferencia de ventas.

In [21]:
sellers = sellers.merge(sales, on='id', how='inner')
sellers.shape

(11, 27)

In [ ]:
sellers['currency_id'].unique()

array(['ARS'], dtype=object)

In [ ]:
sellers.listing_type_id.unique()

array(['gold_special', 'gold_pro'], dtype=object)

In [22]:
value1 = 0.07
value2 = 0.05
condition = np.where(sellers['listing_type_id'] == 'gold_pro', value1, value2)

sellers['q_sales_feb'] = np.round(sellers['February'] * condition, 0)
sellers['q_sales_feb_norm'] = np.where(sellers['available_quantity'] < sellers['q_sales_feb'], sellers.available_quantity, \
                                sellers.q_sales_feb)
sellers['q_sales_mar'] = np.round(sellers['March'] * condition, 0)
sellers['q_sales_mar_norm'] = np.where(sellers['available_quantity'] < sellers['q_sales_mar'], sellers.available_quantity, \
                                sellers.q_sales_mar)
sellers['q_sales_apr'] = np.round(sellers['April'] * condition, 0)
sellers['q_sales_apr_norm'] = np.where(sellers['available_quantity'] < sellers['q_sales_apr'], sellers.available_quantity, \
                                sellers.q_sales_apr)
sellers['sales_feb'] = sellers['q_sales_feb_norm'] * sellers['price']
sellers['sales_mar'] = sellers['q_sales_mar_norm'] * sellers['price']
sellers['sales_apr'] = sellers['q_sales_apr_norm'] * sellers['price']
sellers['total_q_sales'] = sellers['q_sales_feb_norm'] + sellers['q_sales_mar_norm'] + sellers['q_sales_apr_norm']
sellers['total_sales'] = sellers['sales_feb'] + sellers['sales_mar'] + sellers['sales_apr']

sellers.shape


(11, 38)

In [ ]:
sellers.head(2)

,id,title,listing_type_id,permalink,category_id,currency_id,price,original_price,sale_price,available_quantity,official_store_name,stop_time,attributes,seller_id,seller_nickname,installments_quantity,installments_amount,installments_currency_id,name,Condicion del item,Genero,Marca,Modelo,Peso,February,March,April,q_sales_feb,q_sales_feb_norm,q_sales_mar,q_sales_mar_norm,q_sales_apr,q_sales_apr_norm,sales_feb,sales_mar,sales_apr,total_q_sales,total_sales
0,MLA1240020760,Reloj Skmei 9185 Análogico Acero Negro,gold_special,https://www.mercadolibre.com.ar/reloj-skmei-91...,MLA1442,ARS,25919.00,39999.0,None,50,NaN,2042-11-09T04:00:00.000Z,"[{'id': 'AGE_GROUP', 'name': 'Edad', 'value_id...",269998830,DX ELECTRONICA,6.0,5959.21,ARS,Relojes Pulsera,Nuevo,Hombre,Skmei,9185,66 g,4402,4805,789,220.0,50.0,240.0,50.0,39.0,39.0,1295950.00,1295950.00,1010841.00,139.0,3602741.00
1,MLA1660923058,Reloj Curren 8355 Azul Funcional Original De A...,gold_special,https://www.mercadolibre.com.ar/reloj-curren-8...,MLA1442,ARS,68737.33,78779.0,None,1,NaN,2044-02-02T04:00:00.000Z,"[{'id': 'AGE_GROUP', 'name': 'Edad', 'value_id...",83989797,DIBUY COM,6.0,15803.86,ARS,Relojes Pulsera,Nuevo,Hombre,Curren,8355,142 g,1949,2975,2665,97.0,1.0,149.0,1.0,133.0,1.0,68737.33,68737.33,68737.33,3.0,206211.99


Nuestra tabla ahora tiene 38 features. Ya estamos en condiciones de migrar estas tablas hacia nuestra base de datos con el objetivo de realizar los cálculos solicitados.

Migraremos a un csv nuestra tabla final para poder usarla como input más tarde en Tableau.

In [23]:
february = sellers.groupby(['id', 'title', 'listing_type_id', 'permalink', 'price', 'seller_id', 'seller_nickname',\
                    'Genero', 'Marca', 'Modelo', 'Peso']).agg({'February':'sum', 'q_sales_feb_norm': 'sum', 'sales_feb': 'sum'}).reset_index()
february['month'] = 'february'
february = february.rename(columns={'February': 'Visits', 'q_sales_feb_norm':'q_sales', 'sales_feb': 'sales'})

march = sellers.groupby(['id', 'title', 'listing_type_id', 'permalink', 'price', 'seller_id', 'seller_nickname',\
                    'Genero', 'Marca', 'Modelo', 'Peso']).agg({'March':'sum', 'q_sales_mar_norm': 'sum', 'sales_mar': 'sum'}).reset_index()
march['month'] = 'march'
march = march.rename(columns={'March': 'Visits', 'q_sales_mar_norm':'q_sales', 'sales_mar': 'sales'})

april = sellers.groupby(['id', 'title', 'listing_type_id', 'permalink', 'price', 'seller_id', 'seller_nickname',\
                    'Genero', 'Marca', 'Modelo', 'Peso']).agg({'April':'sum', 'q_sales_apr_norm': 'sum', 'sales_apr': 'sum'}).reset_index()
april['month'] = 'april'
april = april.rename(columns={'April': 'Visits', 'q_sales_apr_norm':'q_sales', 'sales_apr': 'sales'})


csv_file = pd.concat([february, march, april])
csv_file.to_csv('/content/sellers.csv', float_format='%.2f', decimal=',', index=False, sep=';')

In [ ]:
csv_file.shape

(33, 15)

## Creación de base de datos con SQLite3

Crearemos nuestra base de datos con `SQLite3` para poder llevar a cabo los análisis correspondientes.

Sin embargo, comenzaremos por guardar nuestro dataframe de python `sellers` dentro de uno nuevo, eliminando campos no solicitados.

In [ ]:
sellers_db = sellers[['id', 'seller_id', 'price', 'q_sales_feb_norm', 'q_sales_mar_norm', 'q_sales_apr_norm', 'sales_feb', \
                      'sales_mar', 'sales_apr']]
sellers_db = sellers_db.rename(columns={'id': 'item_id', 'q_sales_feb_norm': 'q_feb', 'q_sales_mar_norm': 'q_mar', \
                                        'q_sales_apr_norm': 'q_apr'})

sellers_db.head(2)

,item_id,seller_id,price,q_feb,q_mar,q_apr,sales_feb,sales_mar,sales_apr
0,MLA1240020760,269998830,25919.00,50.0,50.0,39.0,1295950.00,1295950.00,1010841.00
1,MLA1660923058,83989797,68737.33,1.0,1.0,1.0,68737.33,68737.33,68737.33


Asimismo, crearemos un segundo dataframe llamado `attributes` que contendrá los datos descriptivos de los productos de nuestra tabla. Estos serán almacenados dentro de una segunda tabla en nuestra base de datos.

In [ ]:
attrib_db = sellers[['id','Condicion del item', 'Genero', 'Marca', 'Modelo', 'Peso']]
attrib_db = attrib_db.rename(columns={'id': 'item_id'})
attrib_db.shape

(11, 6)

Luego crearemos la mentada base de datos y la llamaremos "meli_post.db". A continuación, guardaremos nuestros recientes nuevos dataframes en dos tablas SQL utilizando el método `to_sql`.

In [ ]:
conn = sqlite3.connect('meli_post.db')
cursor = conn.cursor()
sellers_db.to_sql('watch_sales', conn, if_exists='replace', index=False, dtype={'item_id': 'TEXT PRIMARY KEY'})

11

In [ ]:
attrib_db.to_sql('watch_attrib', conn, if_exists='replace', index=False, dtype={'item_id': 'TEXT PRIMARY KEY'})

11

Podemos ver que las bases de datos fueron inicializadas con nuestros once registros.

A continuación, ejecutaremos las consultas sobre ellas.

En primer lugar, un detalle simple de la tabla con todos sus registros mediante un select all.

In [ ]:
query = "SELECT * FROM watch_sales limit 5;"
result = pd.read_sql_query(query, conn)

display(result)

,item_id,seller_id,price,q_feb,q_mar,q_apr,sales_feb,sales_mar,sales_apr
0,MLA1240020760,269998830,25919.00,50.0,50.0,39.0,1295950.00,1295950.00,1010841.00
1,MLA1660923058,83989797,68737.33,1.0,1.0,1.0,68737.33,68737.33,68737.33
2,MLA1618134820,99179853,34999.00,2.0,0.0,1.0,69998.00,0.00,34999.00
3,MLA1702569300,570612785,48798.00,0.0,1.0,1.0,0.00,48798.00,48798.00
4,MLA1361794043,511139715,31349.00,5.0,41.0,100.0,156745.00,1285309.00,3134900.00


In [ ]:
query = "SELECT * FROM watch_attrib limit 5;"
result = pd.read_sql_query(query, conn)

display(result)

,item_id,Condicion del item,Genero,Marca,Modelo,Peso
0,MLA1240020760,Nuevo,Hombre,Skmei,9185,66 g
1,MLA1660923058,Nuevo,Hombre,Curren,8355,142 g
2,MLA1618134820,Nuevo,Hombre,BURK,1251,56 g
3,MLA1702569300,Nuevo,Sin género,Casio,A158WA-1,48 g
4,MLA1361794043,Nuevo,Hombre,Skmei,1068,65 g


Luego analizaremos la composición y el tipo de datos de nuestras tablas. Podemos ver que el campo `item_id` fue asignado como PK en ambas tablas.

In [ ]:
query = "PRAGMA table_info(watch_sales);"
result = pd.read_sql_query(query, conn)

print(result)

   cid       name     type  notnull dflt_value  pk
0    0    item_id     TEXT        0       None   1
1    1  seller_id  INTEGER        0       None   0
2    2      price     REAL        0       None   0
3    3      q_feb     REAL        0       None   0
4    4      q_mar     REAL        0       None   0
5    5      q_apr     REAL        0       None   0
6    6  sales_feb     REAL        0       None   0
7    7  sales_mar     REAL        0       None   0
8    8  sales_apr     REAL        0       None   0


In [ ]:
query = "PRAGMA table_info(watch_attrib);"
result = pd.read_sql_query(query, conn)

print(result)

   cid                name  type  notnull dflt_value  pk
0    0             item_id  TEXT        0       None   1
1    1  Condicion del item  TEXT        0       None   0
2    2              Genero  TEXT        0       None   0
3    3               Marca  TEXT        0       None   0
4    4              Modelo  TEXT        0       None   0
5    5                Peso  TEXT        0       None   0


Por último, antes de realizar las consultas específicas sobre la tabla, las relacionaremos mediante el campo `item_id`.

Para ello crearemos un cursor que nos permita ejecutar la query en SQL.

In [ ]:
c = conn.cursor()
c.execute('CREATE INDEX idx_sales_id ON watch_sales (item_id);')
c.execute('CREATE INDEX idx_attrib_id ON watch_attrib (item_id);')

Finalmente, calcularemos los ingresos totales sobre nuestra base de `sales`.

In [ ]:
query = "select sum(q_feb+q_mar+q_apr) as cantidades_vendidas, sum(sales_feb+sales_mar+sales_apr) as ventas_totales \
from watch_sales;"
result = pd.read_sql_query(query, conn)

print(result)

   cantidades_vendidas  ventas_totales
0                318.0      9955257.48


Y a continuación, el precio promedio de venta por vendedor.

In [ ]:
query = "with table1 as (select seller_id, sum(q_feb+q_mar+q_apr) as cantidades_vendidas, sum(sales_feb+sales_mar+sales_apr) as ventas_totales \
from watch_sales group by seller_id), table2 as (select seller_id, cantidades_vendidas, ventas_totales, (ventas_totales/cantidades_vendidas) \
as promedio_precio_venta \
from table1 group by seller_id order by (ventas_totales/cantidades_vendidas) desc) select * from table2"
result = pd.read_sql_query(query, conn)

print(result)

     seller_id  cantidades_vendidas  ventas_totales  promedio_precio_venta
0    265893742                  3.0       624214.50              208071.50
1     83989797                  3.0       206211.99               68737.33
2    250551537                  3.0       158997.00               52999.00
3    570612785                  2.0        97596.00               48798.00
4     99179853                  3.0       104997.00               34999.00
5   1734129189                 13.0       435500.00               33500.00
6   1619856464                  1.0        32600.00               32600.00
7    511139715                146.0      4576954.00               31349.00
8   1210552456                  3.0        90447.99               30149.33
9    269998830                139.0      3602741.00               25919.00
10    31452355                  2.0        24998.00               12499.00


Guardaremos y cerraremos la conexión creada para ganar eficiencia.

In [ ]:
conn.commit()
conn.close()

## Análisis de resultados obtenidos

A raíz del análisis llevado a cabo, he podido arribar a las siguientes conclusiones:

- En primer lugar, puede observarse que, al menos en la muestra aleatoria que fue considerada, la mayoría de los vendedores no eligió la categoría "gold_pro" para ofrecer sus productos. En la teoría, dicha categoría brindaría mayor propensión a la venta, pues otorga planes de financiación a clientes más prolongados. Solo uno de los once sellers se decidió por esta opción. Si focalizamos nuestro análisis sobre esta empresa en particular (GRUPO_AGG), puede verse que la cantidad de visitas que su publicación recibió durante los últimos 90 días fue de apenas 45, ubicándose en el 7mo lugar si se toma como referencia la nominalidad de las ventas potenciales y en el último lugar si se consideran las visitas propiamente dichas. Sin embargo, este dato contrasta con su tasa de conversión o convertion rate (CR): la misma fue de un 6,67%, lo cual la ubica primera entre todos los sellers sujetos a análisis.
Es destacable considerar, que esta empresa durante el mes de marzo no ofreció productos a la venta, lo cual tiene incidencia no solo sobre las visitas que pudo recibir, sino también sobre la CR.
Si bien este dato es estimativo, podría afirmarse que las condiciones preferenciales que ofrece a sus clientes a la hora del pago, estarían otorgándole a priori una ventaja competitiva frente a sellers del mismo tipo.
- En segundo lugar, y a propósito del convertion rate mencionado, puede verse una mejora creciente en los 3 meses bajo análisis. Mientras en feb/24 el mismo rondaba el 0,42%, en abr/24 escaló al 0,91%. Esto tuvo un correlato, a su vez, sobre las ventas, las cuales se incrementaron un 360% en términos nominales en el mismo período.
- Por último, resulta interesante analizar la correlación entre el precio de venta y las visitas recibidas por los productos. Lo veremos a continuación.

In [91]:
corr_matrix = csv_file[['price', 'Visits']].corr()
corr_matrix['price']

price     1.00000
Visits    0.15629
Name: price, dtype: float64

Podríamos afirmar que el precio de venta no estaría fijado **únicamente** en virtud de las visitas, ya que la correlación es baja. Por decirlo de otra manera: la cantidad de visitas recibidas por un producto no tendría implicancias inmediatas en el aumento del precio del producto, al menos de lo que se desprende del análisis de esta pequeña muestra de casos.

¡Muchas gracias!